In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno  
import re

In [ ]:
# Lecture du fichier Excel
donnees_fifa = pd.read_excel("./fifa_data.xlsx")

In [ ]:
# Nettoyage et traitement de DATA
donnees_fifa.tail()

In [ ]:
# Affichage de la matrice de valeurs manquantes avec noms de colonnes
msno.matrix(donnees_fifa)

In [ ]:
# Récupérer les noms de colonnes
noms_colonnes = donnees_fifa.columns

# Personnaliser l'affichage en ajoutant les noms de colonnes à l'en-tête
plt.title("Matrice de valeurs manquantes avec noms de colonnes")
plt.xlabel("Colonnes")
plt.ylabel("Lignes")
plt.xticks(ticks=range(len(noms_colonnes)), labels=noms_colonnes, rotation=90)

# Afficher le graphique
plt.show()

In [ ]:
# Supprimer les colonnes spécifiées
donnees_fifa = donnees_fifa.drop(columns=["Unnamed: 89","Joined","Contract Valid Until", "Release Clause", "Photo", 
                                          "Flag", "Club Logo","Real Face", "Loaned From", "Special","Unnamed: 0",
                                          "Jersey Number", "Value", "LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW",
                                          "LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", 
                                          "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB",], axis=1)


# Afficher la nouvelle matrice de valeurs manquantes
msno.matrix(donnees_fifa)

In [ ]:
# Recherche des doublons
donnees_fifa.duplicated().sum()

# Supprimer les lignes vides
donnees = donnees_fifa.dropna(subset=['Preferred Foot',"Position"]) 

# Supprimer les lignes dont les postes ne sont pas mentionnés
donnees = donnees.drop(donnees[donnees["Position"] == "No"].index)

# Afficher la nouvelle matrice de valeurs manquantes
msno.matrix(donnees)

In [ ]:
# Afficher les différents postes avec leurs occurrences
occurrences = donnees['Position'].value_counts()

In [ ]:
# Trier les occurrences par ordre décroissant
occurrences = occurrences.sort_values(ascending=False)

plt.figure(figsize=(18, 10))
plt.bar(occurrences.index, occurrences, color='skyblue', edgecolor='black')

# Vérifier le nombre de datas vides dans la colonne Club
print(donnees['Club'].isna().sum())
donnees.info()

In [ ]:
# Substituer les postes du milieu de terrain
donnees.loc[:, "Position"] = donnees["Position"].replace({"CAM":"CM", "CDM":"CM", "RCM":"RM", "RAM":"RM","LCM":"LM", "LAM":"LM"})

# Substituer les postes de la défense 
donnees.loc[:, "Position"] = donnees["Position"].replace({"RCB": "RB", "RWB": "RB","LCB": "LB", "LWB": "LB"})

# Substituer les postes de l'attaque 
donnees.loc[:, "Position"] = donnees["Position"].replace({"CF": "ST","RS": "ST", "LS": "ST","LF": "ST", "RF": "ST"})

# Création de la colonne Groupe de poste
donnees['GroupPositions'] = ""

# Remplir la colonne de façon conditionnelle
donnees.loc[donnees["Position"]== "GK" , "GroupPositions"] = "Goalkeeper"
donnees.loc[donnees["Position"]== "ST" , "GroupPositions"] = "Striker"
donnees.loc[donnees["Position"].isin(["CB","RB","LB"]), "GroupPositions"] = "Defender"
donnees.loc[donnees["Position"].isin(["CM","RM","LM"]), "GroupPositions"] = "Midfielder"
donnees.loc[donnees["Position"].isin(["RDM","LDM"]), "GroupPositions"] = "Defensive Midfielder"
donnees.loc[donnees["Position"].isin(["RW","LW"]), "GroupPositions"] = "Winger"

In [ ]:
# Renommer les abréviations de la colonne Position
position_mapping = {
    'ST': 'Striker',
    'CF': 'Center Forward',
    'LW': 'Left Winger',
    'RW': 'Right Winger',
    'CM': 'Central Midfielder',
    'CDM': 'Central Defensive Midfielder',
    'CAM': 'Central Attacking Midfielder',
    'LM': 'Left Midfielder',
    'RM': 'Right Midfielder',
    'CB': 'Center Back',
    'LB': 'Left Back',
    'RB': 'Right Back',
    'LWB': 'Left Wing Back',
    'RWB': 'Right Wing Back',
    'LS': 'Left Striker',
    'RS': 'Right Striker',
    'LCM': 'Left Central Midfielder',
    'RCM': 'Right Central Midfielder',
    'LDM': 'Left Defensive Midfielder',
    'RDM': 'Right Defensive Midfielder',
    'LF': 'Left Forward',
    'RF': 'Right Forward',
    'LAM': 'Left Attacking Midfielder',
    'RAM': 'Right Attacking Midfielder',
}
donnees['Position'] = donnees['Position'].map(position_mapping)


In [ ]:
# Supprimer l'unité de mesure et convertir en float pour la colonne Crossing
donnees['Crossing'] = pd.to_numeric(donnees['Crossing'], errors='coerce', downcast='integer')

In [ ]:
# Supprimer l'unité de mesure et convertir en float pour la colonne Weight
donnees["Weight"] = donnees["Weight"].astype(str).str.replace("lbs", "").astype(float)

# Convertir le poids en kilogrammes
donnees["Weight"] = donnees["Weight"] * 0.454

In [ ]:
# Supprimer l'unité de mesure et convertir en float pour la colonne Height
donnees["Height"] = donnees["Height"].astype(str).str.extract('(\d+)').astype(float)

# Convertir la hauteur en mètres
donnees["Height"] = donnees["Height"] * 0.3048

In [ ]:
# Repérer les données incorrectes dans la colonne Wage
i = 0
for element in donnees["Wage"]:
    if re.fullmatch("(€[0-9]{1}K|€[0-9]{3}K|€[0-9]{2}K|€[0-9]{1})", str(element)):
        i += 1
    else:
        print(element)

# on extrait les chiffres du motif recherché et on change le type de la colonne 
donnees["Wage"] = donnees["Wage"].str.extract('(\d+)').astype(int) * 1000

In [ ]:
# Repérer les données incorrectes dans la colonne International Reputation
i = 0
for element in donnees["International Reputation"]:
    if re.fullmatch("([0-9]{1})", str(element)):
        i += 1
    else:
        print(element)

# Changer le type de la colonne
donnees['International Reputation'] = pd.to_numeric(donnees['International Reputation'], errors='coerce').fillna(0).astype(int)

# Changer le type de la colonne
donnees['International Reputation'] = donnees['International Reputation'].astype(int)

In [ ]:
# Repérer les données incorrectes dans la colonne Body Type
i = 0
for element in donnees["Body Type"]:
    if element in ["Normal","Lean","Stocky"] : 
        i += 1
    else: 
        print(element)

# Corriger les irrégularités
donnees.loc[~donnees["Body Type"].isin(["Normal","Lean","Stocky"]), "Body Type"] = "Normal"

In [ ]:
# Exportation du dataset clean
donnees.to_excel("../DataSet/data_fifa_clean.xlsx", index=False)